In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow import keras
import scipy
from skimage import io
from scipy import ndimage
from IPython.display import display

%matplotlib inline

In [38]:
sample_df = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
print(sample_df.shape)
sample_df.head()

In [39]:
sample_df.shape

In [40]:
label_list = []
for tag in train_classes.tags.values:
    labels = tag.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [41]:
for label in label_list:
    train_classes[label]= train_classes['tags'].apply(lambda x:1 if label in x.split(' ') else 0)

train_classes.head(5)

In [42]:
train_df = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
train_df.head()

In [43]:
train_df.shape

In [44]:
#print shape of image
img_number = 15
img = io.imread('../input/planets-dataset/planet/planet/train-jpg/train_{}.jpg'.format(img_number))
img.shape

In [45]:
plt.imshow(img)

In [46]:
all_tags = [item for sublist in list(train_df['tags'].apply(lambda row: row.split(" ")).values) for item in sublist]
print('there are {} non-unique total tags in all training images'.format(len(all_tags)))

In [47]:
#checking for unique tags
unique_labels = set()
def append_labels(tags):
    for tag in tags.split():
        unique_labels.add(tag)

train = train_df.copy()
train['tags'].apply(append_labels)
unique_labels = list(unique_labels)
print(unique_labels)

#tags_counted_and_sorted = pd.DataFrame({'tag': all_tags}).groupby('tag').size().reset_index().sort_values(0, ascending=False)
print('There are {} unique tags'.format(len(unique_labels)))
#tags_counted_and_sorted.head()

In [48]:
assert len(train['image_name'].unique()) == train.shape[0]
#one hot encoding labels in the 'train_df'
for tag in unique_labels:
    train[tag] = train['tags'].apply(lambda x: 1 if tag in x.split() else 0)
    
#print head
train['image_name'] = train['image_name'].apply(lambda x: '{}.jpg'.format(x))
train.head()

In [49]:
y_col = list(train.columns[2:])

# initializing an image generator with some data augumentation
image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

# loading images from dataframe
X = image_gen.flow_from_dataframe(dataframe=train,
        directory='../input/planets-dataset/planet/planet/train-jpg', x_col='image_name', y_col=y_col,
       target_size=(128, 128), class_mode='raw', seed=1, batch_size=128)

In [50]:
from glob import glob
image_paths = sorted(glob('../input/planets-dataset/planet/planet/train-jpg/*.jpg'))[0:1000]
image_names = list(map(lambda row: row.split("/")[-1][:-4], image_paths))
image_names[0:10]

In [51]:
plt.figure(figsize=(12,8))
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(plt.imread(image_paths[i]))
    plt.title(str(train_df[train_df.image_name == image_names[i]].tags.values))

In [60]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.metrics import fbeta_score, precision_score, make_scorer, average_precision_score
import cv2
import warnings

n_samples = 5000
rescaled_dim = 20

In [53]:
train_df['split_tags'] = train_df['tags'].map(lambda row: row.split(" "))
lb = MultiLabelBinarizer()
y = lb.fit_transform(train_df['split_tags'])
y = y[:n_samples]
X = np.squeeze(np.array([cv2.resize(plt.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(name)),
                                    (rescaled_dim, rescaled_dim), cv2.INTER_LINEAR).reshape(1, -1) 
                         for name in train_df.head(n_samples)['image_name'].values]))
X = MinMaxScaler().fit_transform(X)

print(X.shape, y.shape, lb.classes_)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =0)

clf = OneVsRestClassifier(LogisticRegression(C=10, penalty='l2'))
clf.fit(X_train, y_train)   
score = fbeta_score(y_test, clf.predict(X_test), beta=2, average=None)
avg_sample_score = fbeta_score(y_test, clf.predict(X_test), beta=2, average='samples')

In [56]:
print('Mean F2 test score {}'.format(avg_sample_score))
print('F2 test scores per tag:')
[(lb.classes_[l], score[l]) for l in score.argsort()[::-1]]

In [64]:
samples= 40669
X_sub = np.squeeze(np.array([cv2.resize(plt.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(name)), (rescaled_dim, rescaled_dim), cv2.INTER_LINEAR).reshape(1, -1) for name in sample_df.head(samples)['image_name'].values]))
X_sub = MinMaxScaler().fit_transform(X_sub)
X_sub.shape

In [62]:
y_sub = clf.predict(X_sub)
all_test_tags = []
for index in range(y_sub.shape[0]):
    all_test_tags.append(' '.join(list(lb.classes_[np.where(y_sub[index, :] == 1)[0]])))
all_test_tags[0:20]

In [63]:
test_images = [plt.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(name)) 
               for name in sample_df.head(20)['image_name'].values]
plt.imshow(test_images[7])

In [61]:
sample_df.head(samples)['tags'] = all_test_tags
sample_df.head()